In [2]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Using TensorFlow backend.


In [3]:
train.shape

(159571, 8)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [5]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [6]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values

In [7]:
vocab = 100000
maxlen = 200
embed_size = 300

In [8]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [9]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [10]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [11]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [35]:
num_filters = 32

inp = Input(shape=(maxlen, ))
x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
x = SpatialDropout1D(0.5)(x)
x = Reshape((maxlen, embed_size, 1))(x)

conv_0 = Conv2D(num_filters, kernel_size=(2, embed_size), activation='relu')(x)
maxpool_0 = MaxPool2D(pool_size=(maxlen-1, 1))(conv_0)
conv_1 = Conv2D(num_filters, kernel_size=(4, embed_size), activation='relu')(x)
maxpool_1 = MaxPool2D(pool_size=(maxlen-3, 1))(conv_1)
conv_2 = Conv2D(num_filters, kernel_size=(6, embed_size), activation='relu')(x)
maxpool_2 = MaxPool2D(pool_size=(maxlen-5, 1))(conv_2)
conv_3 = Conv2D(num_filters, kernel_size=(8, embed_size), activation='relu')(x)
maxpool_3 = MaxPool2D(pool_size=(maxlen-7, 1))(conv_3)
conv_4 = Conv2D(num_filters, kernel_size=(10, embed_size), activation='relu')(x)
maxpool_4 = MaxPool2D(pool_size=(maxlen-9, 1))(conv_4)

y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4])
y = Flatten()(y)
y = Dropout(0.1)(y)

outp = Dense(6, activation="sigmoid")(y)

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_15 (InputLayer)            (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_15 (Embedding)         (None, 200, 300)      30000000                                     
____________________________________________________________________________________________________
spatial_dropout1d_15 (SpatialDro (None, 200, 300)      0                                            
____________________________________________________________________________________________________
reshape_15 (Reshape)             (None, 200, 300, 1)   0                                            
___________________________________________________________________________________________

In [36]:
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.80, random_state=8)

In [37]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [38]:
from keras.callbacks import ModelCheckpoint
batch_size=256
epochs = 3
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.pre_trained.conv2.hdf5', verbose=1, save_best_only=True)
RocAuc = RocAuc()

In [39]:
h = model.fit(x_trainS, y_trainS, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
                 callbacks=[checkpointer, RocAuc], verbose=2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/3
Epoch 00000: val_loss improved from inf to 0.04980, saving model to saved_models/weights.best.pre_trained.conv2.hdf5

 roc_auc - epoch: 1 - score: 0.973919 

1236s - loss: 0.0779 - acc: 0.9762 - val_loss: 0.0498 - val_acc: 0.9819
Epoch 2/3
Epoch 00001: val_loss improved from 0.04980 to 0.04654, saving model to saved_models/weights.best.pre_trained.conv2.hdf5

 roc_auc - epoch: 2 - score: 0.984535 

1248s - loss: 0.0533 - acc: 0.9809 - val_loss: 0.0465 - val_acc: 0.9827
Epoch 3/3
Epoch 00002: val_loss did not improve

 roc_auc - epoch: 3 - score: 0.986953 

1239s - loss: 0.0493 - acc: 0.9819 - val_loss: 0.0466 - val_acc: 0.9828
